In [ ]:

from commutazzio.filtration import CLFiltration
from commutazzio.compute import CLInvariants
from commutazzio.filtration import CLFiltrationDB
# from commutazzio.plot import ComplementaryTrianglesPlot as Visualizer1
import random
import numpy as np
from echoflare import send_finish_message
from commutazzio.utils import filepath_generator

import pandas as pd
pd.options.display.max_rows = None
pd.set_option('display.max_columns', None)
import os
import glob
from commutazzio.statistics import CL4_NIScores, CLN_NIScores  


In [ ]:
# Read the dataframes
cl4_df = pd.read_pickle('cl4_df_pc_large.pkl')
cln_df = pd.read_pickle('cln_df_pc_large.pkl')
sample_4=cl4_df.sample(10000)
sample_n = cln_df.sample(10000)
Is=[f"I{_}" for _ in range(1, 56)]
Ns=[f"N{_}" for _ in range(1, 22)]
Ls=Is+Ns
# to dict, join Is and Ns
mult_dict=sample_4.iloc[0][Is+Ns].to_dict()
# create a dict with keys Ls and random positive integers as values
mult_dict={k:random.randint(1, 100) for k in Ls}


In [ ]:
# for i in range(len(sample_n)):
#     # return the first i with lines containing meaningful data
#     lines = sample_n.iloc[i]['lines']
#     if lines != ',x0,y0,x1,y1,multiplicity\n':
#         print(lines)
#         print(i)
#         break

# sample_n.iloc[i]['lines']
# dots,lines=sample_n.iloc[i]['dots'],sample_n.iloc[i]['lines']
# dots=',x,y,multiplicity,area\n0,1,1,1,U\n1,2,2,1,U\n2,4,4,1,U\n3,6,6,1,U\n4,6,8,1,U\n5,9,6,1,D\n6,7,7,1,U\n7,8,7,1,D\n8,11,11,1,U\n9,12,12,1,U\n'
# lines=',x0,y0,x1,y1,multiplicity\n0,9,6,6,8,1\n1,8,7,6,7,-1\n2,8,7,6,8,1\n3,8,7,7,7,1\n4,9,7,6,7,1\n5,9,7,6,8,-1\n'
# lines=',x0,y0,x1,y1,multiplicity\n'
# aa=CLN_NIScores(dots,lines)
# # aa.agg_total
# # aa._numerator_numba
# # aa.ns_c_prime

def cl4_add_cols(df):
    # for each row, extract Ls, create a CL4_NIScores instance, 
    # compute  ns_const and ns_logistic(i) for i from 0 to 8
    # get dots and rows, create a CLN_NIScores instance,
    # compute ns_c_prime
    # add the results to the dataframe
    def nScores(row):
        mult_dict=row[Ls].to_dict()
        obj=CL4_NIScores(mult_dict)
        return obj.ns_const, obj.ns_logistic(0), obj.ns_logistic(1), obj.ns_logistic(2), obj.ns_logistic(3), obj.ns_logistic(4), obj.ns_logistic(5), obj.ns_logistic(6), obj.ns_logistic(7), obj.ns_logistic(8)
    df[['ns_const']+['ns_logistic_'+str(i) for i in range(9)]]=df.apply(nScores, axis=1, result_type='expand')
    def nSprime(row):
        dots,lines=row['dots'],row['lines']
        obj=CLN_NIScores(dots,lines)
        return obj.ns_c_prime
    df['ns_c_prime']=df.apply(nSprime, axis=1)
    return df
def cln_add_cols(df):
    def nSprime(row):
        dots,lines=row['dots'],row['lines']
        obj=CLN_NIScores(dots,lines)
        return obj.ns_c_prime
    df['ns_c_prime']=df.apply(nSprime, axis=1)
    # define capture ratio
    def capture_ratio(row):
        numerator=row['ladder_length']
        denominator=row['critical_radii_number']
        return numerator/denominator
    df['capture_ratio']=df.apply(capture_ratio, axis=1)
    return df


In [ ]:
cl4_pc_decorated=cl4_add_cols(cl4_df)
cln_pc_decorated=cln_add_cols(cln_df)
cl4_pc_decorated.to_pickle('cl4_pc_large_decorated.pkl')
cln_pc_decorated.to_pickle('cln_pc_large_decorated.pkl')